In [1]:
#import "Movie Lens Small Latest Dataset" dataset

import pandas as pd
import numpy as np
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

links= pd.read_csv('./links.csv')
movies=pd.read_csv('./movies.csv')
ratings=pd.read_csv('./ratings.csv')
tags=pd.read_csv('./tags.csv')

dataset=movies.merge(ratings,on='movieId').merge(tags,on='movieId').merge(links,on='movieId')
dataset.head()

,movieId,title,genres,userId_x,rating,timestamp_x,userId_y,tag,timestamp_y,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,336,pixar,1139045764,114709,862.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,474,pixar,1137206825,114709,862.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,567,fun,1525286013,114709,862.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,336,pixar,1139045764,114709,862.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,474,pixar,1137206825,114709,862.0


In [2]:
to_drop=['title','genres','timestamp_x','timestamp_y','userId_y','imdbId','tmdbId']

dataset.drop(columns=to_drop,inplace=True)
print(dataset.describe())
dataset.head()

             movieId       userId_x         rating
count  233213.000000  233213.000000  233213.000000
mean    12319.999443     309.688191       3.966535
std     28243.919401     178.206387       0.968637
min         1.000000       1.000000       0.500000
25%       296.000000     156.000000       3.500000
50%      1198.000000     309.000000       4.000000
75%      4638.000000     460.000000       5.000000
max    193565.000000     610.000000       5.000000


,movieId,userId_x,rating,tag
0,1,1,4.0,pixar
1,1,1,4.0,pixar
2,1,1,4.0,fun
3,1,5,4.0,pixar
4,1,5,4.0,pixar


In [3]:
dataset=pd.get_dummies(dataset) #encode the catergorical data

print(dataset)
dataset.isnull().sum()# check the number of missing data cells

        movieId  userId_x  rating  tag_"artsy"  \
0             1         1     4.0            0   
1             1         1     4.0            0   
2             1         1     4.0            0   
3             1         5     4.0            0   
4             1         5     4.0            0   
...         ...       ...     ...          ...   
233208   187595       586     5.0            0   
233209   193565       184     3.5            0   
233210   193565       184     3.5            0   
233211   193565       184     3.5            0   
233212   193565       184     3.5            0   

        tag_06 Oscar Nominated Best Movie - Animation  tag_1900s  tag_1920s  \
0                                                   0          0          0   
1                                                   0          0          0   
2                                                   0          0          0   
3                                                   0          0          0   
4   

movieId                                          0
userId_x                                         0
rating                                           0
tag_"artsy"                                      0
tag_06 Oscar Nominated Best Movie - Animation    0
                                                ..
tag_wry                                          0
tag_younger men                                  0
tag_zither                                       0
tag_zoe kazan                                    0
tag_zombies                                      0
Length: 1587, dtype: int64

In [4]:
#Divide data into test,train and validation
#train dataset
train_dataset = dataset.sample(frac=0.9,random_state=0) #90% of the dataset
test_dataset=dataset.drop(train_dataset.index) #10% of the Dataset


#seperate labels
train_labels = train_dataset.pop('rating')
test_labels = test_dataset.pop('rating')

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

#scale the features
sc = StandardScaler()
train_dataset = sc.fit_transform(train_dataset)
test_dataset = sc.fit_transform(test_dataset)

#train model
regressor = RandomForestRegressor(n_estimators=10, random_state=0)
regressor.fit(train_dataset, train_labels)

#predict ratings on test data
predicted_labels = regressor.predict(test_dataset)

In [6]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [7]:
actual_labels=np.array(test_labels)

print('Mean Absolute Error:', metrics.mean_absolute_error(actual_labels, predicted_labels))
print('Mean Squared Error:', metrics.mean_squared_error(actual_labels, predicted_labels))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(actual_labels, predicted_labels)))
print('Mean Average Percentage Error: ',mean_absolute_percentage_error(actual_labels,predicted_labels))

Mean Absolute Error: 0.8956691394022556
Mean Squared Error: 1.3245358260794993
Root Mean Squared Error: 1.15088480139391
Mean Average Percentage Error:  31.136428404673765


In [8]:
print('Size of Test Labels',actual_labels.size)
print('Size of Predicted Labels',predicted_labels.size)

#create a new dataframe
predicted_movies=pd.DataFrame({'Actual':actual_labels,'Predicted':predicted_labels}).reset_index()

Size of Test Labels 23321
Size of Predicted Labels 23321


In [9]:
#print Table of Test Dataset
predicted_movies.head(20)

,index,Actual,Predicted
0,0,4.0,4.90
1,1,2.5,5.00
2,2,3.5,3.85
3,3,5.0,3.55
4,4,3.0,3.55
5,5,3.0,3.90
6,6,4.0,3.90
7,7,3.0,2.80
8,8,2.5,4.40
9,9,4.5,4.35
